In [23]:
!pip install geopandas matplotlib

import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy import stats

%matplotlib inline

In [0]:
def impute_outliers(df, column, non_statistics_columns):
  if column not in non_statistics_columns:
    # replaces outliers with maximum or minimum non outlier
    max_non_outlier = df.loc[(np.abs(stats.zscore(df[column])) < 3), column].max()
    min_non_outlier = df.loc[(np.abs(stats.zscore(df[column])) < 3), column].min()

    df.loc[(stats.zscore(df[column]) > 3), column] = max_non_outlier
    df.loc[(stats.zscore(df[column]) < -3), column] = min_non_outlier

In [0]:
data = pd.read_csv('drive/My Drive/Miscellaneous/Projects/ohio-inequity/dataset.csv')
non_statistic_columns = ['IRN', 'School District', 'County']
for column in data.columns:
  impute_outliers(data, column, non_statistic_columns)

geometry_data = gpd.read_file('drive/My Drive/Miscellaneous/Projects/ohio-inequity/State_Wide_School_Districts.geojson')
geometry_data = geometry_data.drop(index=geometry_data.index[0:1])
geometry_data = geometry_data.reset_index(drop=True)
geometry_data = geometry_data[['ODE_IRN', 'Shape_Leng', 'geometry']]
geometry_data['ODE_IRN'] = geometry_data['ODE_IRN'].astype(int)

pd.DataFrame.to_csv(data, 'final_dataset.csv')
geometry_data.to_file("school_districts.geojson", driver='GeoJSON')